# Setup

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

meter_file = 'src_data/_MeteringLineValue__202402102328.csv'
tarif_file = 'src_data/07-02-2024_2500_NP_99.csv'
output_dir_illustrations = './sink_illustrations/'
output_dir_data = './sink_illustrations/'
os.makedirs(output_dir_illustrations, exist_ok=True)
os.makedirs(output_dir_data, exist_ok=True)
meter_data = pd.read_csv(meter_file)
tarif_data = pd.read_csv(tarif_file)

# Exploration
Using data wrangler in VS Code for general exploration reg. distribution, nulls, types etc.

In [2]:
# Visualization setup
sns.set_theme(style="whitegrid")

# Histograms for numerical columns
numerical_cols_1 = meter_data.select_dtypes(include=[np.number]).columns
numerical_cols_2 = tarif_data.select_dtypes(include=[np.number]).columns

for col in numerical_cols_1:
    plt.figure(figsize=(8, 4))
    sns.histplot(meter_data[col].dropna(), kde=True)
    plt.title(f'Distribution of {col} in File 1')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.savefig(os.path.join(output_dir_illustrations, f'meter_data_distribution_{col}.png'))
    plt.close()

for col in numerical_cols_2:
    plt.figure(figsize=(8, 4))
    sns.histplot(tarif_data[col].dropna(), kde=True)
    plt.title(f'Distribution of {col} in File 2')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.savefig(os.path.join(output_dir_illustrations, f'tarif_data_distribution_{col}.png'))
    plt.close()

# Count plots for categorical columns
categorical_cols_1 = meter_data.select_dtypes(include=[object]).columns
categorical_cols_2 = tarif_data.select_dtypes(include=[object]).columns

for col in categorical_cols_1:
    plt.figure(figsize=(8, 4))
    sns.countplot(y=meter_data[col])
    plt.title(f'Distribution of {col} in File 1')
    plt.xlabel('Count')
    plt.ylabel(col)
    plt.savefig(os.path.join(output_dir_illustrations, f'meter_data_distribution_{col}.png'))
    plt.close()

for col in categorical_cols_2:
    plt.figure(figsize=(8, 4))
    sns.countplot(y=tarif_data[col])
    plt.title(f'Distribution of {col} in File 2')
    plt.xlabel('Count')
    plt.ylabel(col)
    plt.savefig(os.path.join(output_dir_illustrations, f'tarif_data_distribution_{col}.png'))
    plt.close()